In [2]:
import time
import os

import requests # used for non-javascript sites
from selenium import webdriver # used for javascript sites
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import bs4
from collections import Counter 

import json
import mistral

### functions

In [3]:
def get_html(website):
    html = requests.get(website).text

    if html and "JavaScript" in html: # most frameworks will say "JS is required"
        options = webdriver.ChromeOptions()
        options.add_argument('--headless=new')
        driver = webdriver.Chrome(options=options)
        driver.get(website)
        
        time.sleep(10) # wait for any js elements to load

        html = driver.page_source
        driver.quit()
        
    return html



In [4]:
def find_next_page(html):
    soup = bs4.BeautifulSoup(html, 'html.parser')
    paginate = soup.find_all('a')
    for link in paginate:
        if 'next' in link.text.lower():
           return link['href']
    return None


In [5]:
def is_top_level_class(element, class_names):
    for parent in element.parents:
        if hasattr(parent, 'get') and parent.get('class'): # a bs4 element will have a get method
            if any(cls in parent.get('class', []) for cls in class_names):
                return False
    return True



In [6]:
def parse_html(html):
    soup = bs4.BeautifulSoup(html, 'html.parser')

    class_counter = Counter()

    for element in soup.find_all(class_=True):
        for class_name in element['class']: 
            class_counter[class_name] += 1 

    quantity_counter = Counter()
    for class_name, count in class_counter.most_common(20): 
        quantity_counter[count] += 1
        
    data_classes = []
    for count in quantity_counter.most_common(1):
        data_classes.extend([class_name for class_name, c in class_counter.items() if c == count[0]])

    print("Data-related classes:", data_classes)
    
    filtered_elements = [
        el for el in soup.find_all(class_=data_classes)
        if is_top_level_class(el, data_classes)
    ]
    elements = filtered_elements if len(filtered_elements) > 0 else [soup.body]
    return elements

In [7]:
def remove_unwanted_text(response):
    # most LLM's will return json as a code block string
    if "```json" in response:
        response = response.split("```json")[1]
        response = response.split("```")[0]
    return response

In [8]:
def convert_to_json(elements):
    html_elements = "\n".join(str(el) for el in elements)
    chat_response = mistral.chat(html_elements).choices[0].message.content
    json_data = json.loads(remove_unwanted_text(chat_response))
    return json_data

In [9]:
def convert_to_json_manual(elements):
    # manual conversion with knowledge of HTML structure
    json_elements = []
    for el in elements:
        json_element = {
            "date": el.find(class_="relativetime").text if el.find(class_="relativetime") else None,
            "votes": (
                el.find_all(class_="s-post-summary--stats-item-number")[0].text
                if len(el.find_all(class_="s-post-summary--stats-item-number")) > 0
                else None
            ),
            "answers": (
                el.find_all(class_="s-post-summary--stats-item-number")[1].text
                if len(el.find_all(class_="s-post-summary--stats-item-number")) > 1
                else None
            ),
            "views": (
                el.find_all(class_="s-post-summary--stats-item-number")[2].text
                if len(el.find_all(class_="s-post-summary--stats-item-number")) > 2
                else None
            ),
            "tags": [tag.text for tag in el.find_all(class_="post-tag")],
            "user": {
                "name": el.find(class_="s-user-card--link").text if el.find(class_="s-user-card--link") else None,
                "reputation": (
                    el.find(class_="s-user-card--rep").text
                    if el.find(class_="s-user-card--rep")
                    else None
                ),
            },
            "excerpt": el.find(class_="s-post-summary--content-excerpt").text if el.find(class_="s-post-summary--content-excerpt") else None,
            "title": (lambda: el.find(class_="s-post-summary--content-title").a.text)() if el.find(class_="s-post-summary--content-title") and el.find(class_="s-post-summary--content-title").a else None,
            "page_content": get_html(f"https://philosophy.stackexchange.com{el.find(class_='s-post-summary--content-title').a['href']}") if el.find(class_="s-post-summary--content-title") and el.find(class_="s-post-summary--content-title").a and el.find(class_="s-post-summary--content-title").a.get('href') else None
        }
        json_elements.append(json_element)
    return json_elements

In [10]:
class Scraper:
    def __init__(self, website, try_paginate=False):
        self.website = website
        self.html = None
        self.elements = None
        self.json_data = None
        self.try_paginate = try_paginate

    def run(self):
        if self.try_paginate:
            print("Paginating")
            self.run_pages()
        else:
            self.run_single() 
    
    def run_pages(self):
        page_url = self.website
        i = 0
        name = self.website.replace('https://', '').replace('http://', '').replace('/', '_')
        while page_url:
            print(f"Scraping {page_url}")
            self.html = get_html(page_url)
            self.elements = parse_html(self.html)
            with open(f"loop_html/{name}{i}.html", "w", encoding="utf-8") as f:
                f.write(self.html)
            self.json_data = convert_to_json(self.elements)
            with open(f"loop_json/{name}{i}.json", "w", encoding="utf-8") as f:
                json.dump(self.json_data, f, indent=2)
            next_page = find_next_page(self.html)
            if next_page:
                if next_page.startswith('/'):
                    page_url = f"{self.website}{next_page}"
                elif next_page.startswith('http'):
                    page_url = next_page
                else:
                    page_url = f"{page_url.rsplit('/', 1)[0]}/{next_page}"
                i += 1 
            else:
                break

    def run_single(self):
        self.html = get_html(self.website)
        self.elements = parse_html(self.html)
        self.json_data = convert_to_json(self.elements)
        return self.json_data

### extracting html

In [1]:
website = "https://philosophy.stackexchange.com/questions?tab=newest&pagesize=50"
html = get_html(website)
print(html)

NameError: name 'get_html' is not defined

### parsing html


In [ ]:
soup = bs4.BeautifulSoup(html, 'html.parser')

class_counter = Counter()

for element in soup.find_all(class_=True):
    for class_name in element['class']: 
        class_counter[class_name] += 1 

quantity_counter = Counter()
for class_name, count in class_counter.most_common(20): 
    quantity_counter[count] += 1
    
data_classes = []
for count in quantity_counter.most_common(1):
    data_classes.extend([class_name for class_name, c in class_counter.items() if c == count[0]])

print("Data-related classes:", data_classes)

filtered_elements = [
    el for el in soup.find_all(class_=data_classes)
    if is_top_level_class(el, data_classes)
]

elements = filtered_elements if len(filtered_elements) > 0 else [soup.body]


Data-related classes: ['s-post-summary', 'js-post-summary', 's-post-summary--stats', 'js-post-summary-stats', 's-post-summary--stats-item__emphasized', 's-post-summary--content', 's-post-summary--content-title', 's-post-summary--content-excerpt', 's-post-summary--meta', 's-post-summary--meta-tags', 'd-inline-block', 'tags', 'js-tags', 'ml0', 'list-ls-none', 'js-post-tag-list-wrapper', 's-user-card', 's-user-card__minimal', 's-avatar', 's-avatar__16', 's-user-card--avatar', 'gravatar-wrapper-16', 's-avatar--image', 's-user-card--info', 's-user-card--link', 's-user-card--awards', 's-user-card--rep', 'todo-no-class-here', 's-user-card--time', 'relativetime']


### converting html to json

In [ ]:
html_elements = "\n".join(str(el) for el in elements)
chat_response = mistral.chat(html_elements).choices[0].message.content


KeyboardInterrupt: 

In [ ]:
json_data = json.loads(remove_unwanted_text(chat_response))

In [ ]:
with open("data.json", "w") as f:
    json.dump(json_data, f, indent=2)

### convert_to_json()

## all together now to loop it

In [12]:
site = "https://philosophy.stackexchange.com/questions?tab=newest&page=1&pagesize=50"

for i in range(451):
    site = f"https://philosophy.stackexchange.com/questions?tab=newest&page={i+1}&pagesize=50"
    html = get_html(site)
    with open(f"loop_html/philosophy_questions_page_{i+1}.html", "w", encoding="utf-8") as f:
        f.write(html)

KeyboardInterrupt: 

In [ ]:

for i in range(len(os.listdir("loop_html"))):
    with open(f"loop_html/philosophy_questions_page_{i+1}.html", "r", encoding="utf-8") as f:
        html = f.read()
    elements = parse_html(html)
    json_data = convert_to_json_manual(elements)
    with open(f"loop_json/philosophy_questions_page_{i+1}.json", "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2)

Data-related classes: ['s-post-summary', 'js-post-summary', 's-post-summary--stats', 'js-post-summary-stats', 's-post-summary--stats-item__emphasized', 's-post-summary--content', 's-post-summary--content-title', 's-post-summary--content-excerpt', 's-post-summary--meta', 's-post-summary--meta-tags', 'd-inline-block', 'tags', 'js-tags', 'ml0', 'list-ls-none', 'js-post-tag-list-wrapper', 's-user-card', 's-user-card__minimal', 's-avatar', 's-avatar__16', 's-user-card--avatar', 'gravatar-wrapper-16', 's-avatar--image', 's-user-card--info', 's-user-card--link', 's-user-card--awards', 's-user-card--rep', 'todo-no-class-here', 's-user-card--time', 'relativetime']
Data-related classes: ['s-post-summary', 'js-post-summary', 's-post-summary--stats', 'js-post-summary-stats', 's-post-summary--stats-item__emphasized', 's-post-summary--content', 's-post-summary--content-title', 's-post-summary--content-excerpt', 's-post-summary--meta', 's-post-summary--meta-tags', 'd-inline-block', 'tags', 'js-tags',

The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry


Data-related classes: ['s-post-summary', 'js-post-summary', 's-post-summary--stats', 'js-post-summary-stats', 's-post-summary--stats-item__emphasized', 's-post-summary--content', 's-post-summary--content-title', 's-post-summary--content-excerpt', 's-post-summary--meta', 's-post-summary--meta-tags', 'd-inline-block', 'tags', 'js-tags', 'ml0', 'list-ls-none', 'js-post-tag-list-wrapper', 's-user-card', 's-user-card__minimal', 's-avatar', 's-avatar__16', 's-user-card--avatar', 'gravatar-wrapper-16', 's-avatar--image', 's-user-card--info', 's-user-card--link', 's-user-card--awards', 's-user-card--rep', 'todo-no-class-here', 's-user-card--time', 'relativetime']


The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry


Data-related classes: ['s-post-summary', 'js-post-summary', 's-post-summary--stats', 'js-post-summary-stats', 's-post-summary--stats-item__emphasized', 's-post-summary--content', 's-post-summary--content-title', 's-post-summary--content-excerpt', 's-post-summary--meta', 's-post-summary--meta-tags', 'd-inline-block', 'tags', 'js-tags', 'ml0', 'list-ls-none', 'js-post-tag-list-wrapper', 's-user-card', 's-user-card__minimal', 's-avatar', 's-avatar__16', 's-user-card--avatar', 'gravatar-wrapper-16', 's-avatar--image', 's-user-card--info', 's-user-card--link', 's-user-card--time', 'relativetime']


...

The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry


Data-related classes: ['s-post-summary', 'js-post-summary', 's-post-summary--stats', 'js-post-summary-stats', 's-post-summary--stats-item__emphasized', 's-post-summary--content', 's-post-summary--content-title', 's-post-summary--content-excerpt', 's-post-summary--meta', 's-post-summary--meta-tags', 'd-inline-block', 'tags', 'js-tags', 'ml0', 'list-ls-none', 'js-post-tag-list-wrapper', 's-user-card', 's-user-card__minimal', 's-avatar', 's-avatar__16', 's-user-card--avatar', 'gravatar-wrapper-16', 's-avatar--image', 's-user-card--info', 's-user-card--link', 's-user-card--time', 'relativetime']


The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry


Data-related classes: ['s-post-summary', 'js-post-summary', 's-post-summary--stats', 'js-post-summary-stats', 's-post-summary--stats-item__emphasized', 'has-answers', 's-post-summary--content', 's-post-summary--content-title', 's-post-summary--content-excerpt', 's-post-summary--meta', 's-post-summary--meta-tags', 'd-inline-block', 'tags', 'js-tags', 'ml0', 'list-ls-none', 'js-post-tag-list-wrapper', 's-user-card', 's-user-card__minimal', 's-avatar', 's-avatar__16', 's-user-card--avatar', 'gravatar-wrapper-16', 's-avatar--image', 's-user-card--info', 's-user-card--link', 's-user-card--time', 'relativetime']


The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (138.0.7204.183) detected in PATH at C:\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry


In [13]:
json_data = {"questions": []}

for i in range(len(os.listdir("loop_json"))):
    with open(f"loop_json/philosophy_questions_page_{i+1}.json", "r", encoding="utf-8") as f:
        page_data = json.load(f)
    json_data["questions"].extend(page_data)

with open("philosophy_questions_all.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=2)

## testing/example

In [11]:
site = "https://books.toscrape.com/"
json_data = Scraper(site, try_paginate=True).run()
with open("books.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=2)

Paginating
Scraping https://books.toscrape.com/
Data-related classes: ['col-xs-6', 'col-lg-3', 'product_pod', 'image_container', 'thumbnail', 'star-rating', 'product_price', 'price_color', 'instock', 'availability', 'icon-ok', 'btn', 'btn-primary', 'btn-block']
Scraping https://books.toscrape.com/catalogue/page-2.html
Data-related classes: ['col-xs-6', 'col-lg-3', 'product_pod', 'image_container', 'thumbnail', 'star-rating', 'product_price', 'price_color', 'instock', 'availability', 'icon-ok', 'btn', 'btn-primary', 'btn-block']
Scraping https://books.toscrape.com/catalogue/page-3.html
Data-related classes: ['col-xs-6', 'col-lg-3', 'product_pod', 'image_container', 'thumbnail', 'star-rating', 'product_price', 'price_color', 'instock', 'availability', 'icon-ok', 'btn', 'btn-primary', 'btn-block']
Scraping https://books.toscrape.com/catalogue/page-4.html
Data-related classes: ['col-xs-6', 'col-lg-3', 'product_pod', 'image_container', 'thumbnail', 'star-rating', 'product_price', 'price_col